# Azure AI Agents wey get Model Context Protocol (MCP) Support

Dis notebook dey show how you fit use Azure AI Agents wey get Model Context Protocol (MCP) tools to build one smart agent wey fit use external MCP servers to get better abilities.


## Install Di NuGet Packages Wey You Go Need

First, we go need install di Azure AI Agents Persistent package wey dey provide di main functionality to take work wit Azure AI Agents.


## Benefits of Keyless Authentication

Dis notebook dey show how **keyless authentication** fit work, wey get plenty benefits:
- ✅ **No need to dey manage API keys** - E dey use Azure identity-based authentication
- ✅ **Better security** - No secret go dey for code or configuration
- ✅ **Automatic credential rotation** - Azure go handle di credential lifecycle
- ✅ **Role-based access** - E dey use Azure RBAC to give permissions wey fit di role

Di `DefaultAzureCredential` go automatically use di best credential source wey dey available:
1. Managed Identity (if you dey run am for Azure)
2. Azure CLI credentials (when you dey develop)
3. Visual Studio credentials
4. Environment variables (if dem don configure am)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Instal di Azure Identity package wey go help you do authentication wit Azure services using DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Import Di Namespaces Wey You Need

Carry di namespaces wey you need for Azure AI Agents and Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Configure Azure AI Agent Client (Keyless Authentication)

Set up di configuration variables and create di PersistentAgentsClient wey go use **keyless authentication**:
- **projectEndpoint**: Di Azure AI Foundry project endpoint
- **modelDeploymentName**: Di name of di AI model wey dem don deploy (GPT-4.1 nano)
- **mcpServerUrl**: Di URL of di MCP server (Microsoft Learn API)
- **mcpServerLabel**: Label wey go help identify di MCP server
- **DefaultAzureCredential**: E dey use managed identity, Azure CLI, or other credential sources (no need API keys)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Create MCP Tool Definition

Make MCP tool definition wey go connect to di Microsoft Learn MCP server. Dis one go allow di agent fit access Microsoft Learn content and documentation.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Create di AI Agent

Make one AI agent wey go dey permanent wit di model and MCP tools wey dem don specify. Di agent go dey set up wit di following:

- Di GPT-4.1 nano model  
- Instructions say make e use MCP tools to take help  
- Access to di Microsoft Learn MCP server  


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Create Thread and Send Message

Make conversation thread and send message to user wey dey ask about di difference between Azure OpenAI and OpenAI. Dis one go test di agent ability to use MCP tools to give correct information.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Set MCP Tool Resources (Keyless)

Arrange di MCP tool resources. If you wan do am truly keyless, you fit comot custom headers if di MCP server dey support Azure identity-based authentication. Di example wey dey below go show how you fit add headers if e dey necessary, but for keyless setup, you no go need dem.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Start Agent Run

Create and start run wey go process di user message. Di agent go use di MCP tools and resources wey dem configure to generate response.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Monitor Run and Handle Tool Approvals (Keyless)

Dey check how di agent run dey go and handle any tool approval wey dem need. Dis loop:
1. E go wait make di run finish or if e need action
2. E go approve MCP tool calls automatic if dem need am
3. For keyless authentication, headers no go dey necessary if di MCP server support Azure identity


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Show Conversation Results

Bring out and show all di messages wey dey inside di thread, make e show di user question and di agent reply. Di messages go show for di order wey dem happen, wit time wey dem take happen and role wey dem play.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Dis dokyument don use AI transleto service [Co-op Translator](https://github.com/Azure/co-op-translator) do di translation. Even as we dey try make am correct, abeg sabi say machine translation fit get mistake or no dey accurate well. Di original dokyument for di language wey dem write am first na di main source wey you go trust. For important mata, e good make professional human translator check am. We no go fit take blame for any misunderstanding or wrong interpretation wey fit happen because you use dis translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
